In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
import pandas as pd
import os, json

def save_json(file, data):
    file_path = f"{file}.json"
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            existing_data = json.load(f)
        existing_data.extend(data)
        with open(file_path, "w") as f:
            f.write(json.dumps(existing_data, indent=2))
    else:
        try:
            with open(file_path, "w") as f:
                f.write(json.dumps(data, indent=2))
        except Exception as e:
            print(e)

In [ ]:
import requests
from bs4 import BeautifulSoup
import json


headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36'}

# Replace with your target URL
url = "https://biblicalcounseling.com/resource-library/topic-index/"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
categories = {}
category_sections = soup.find_all('li', class_='collection__item')
for section in category_sections:
    category_name = section.find('a',class_="collection__item--title").text.strip()        
    links = section.find_all('li', class_='term_child')
    # Store links for this category
    categories[category_name] = [{
        'title': link.text.strip(),
        'url': link.find("a")["href"]
    } for link in links]

In [65]:
def scrape_content(link, category, subcategory):
    # url = "https://biblicalcounseling.com"
    # print(link)

    response = requests.get(f'{link}', headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    header_component = soup.find("div", class_= "card-content")
    title = header_component.find("h1").text.strip() if header_component.find("h1") else " " 
    subtitle = header_component.find("h3").text.strip() if header_component.find("h3") else " " 
    question = header_component.find("p").text.strip() if header_component.find("p") else " " 
    authors = header_component.find("div", class_="authors-wrapper").text.strip() if header_component.find("div", class_="authors-wrapper") else " "
    content = soup.find("div", class_="article-content").text.strip()
    source = url

    return {
            "title": title,
            "subtitle": subtitle,
            "author": authors,
            "category": category,
            "subcategory": subcategory,
            "question": question,
            "body": content,
            "source": source
        }

In [31]:
categories

{}

In [66]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36'}
base_url = "https://biblicalcounseling.com"

data_list_dict = []

for k, v in  list(categories.items())[1:2]:
    for vals in v:
        response = requests.get(f'{base_url}{vals['url']}', headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find("div", class_="card-list")
        articles_links = articles.find_all("a", class_="overlink")
        articles_links = [f"{link['href']}" for link in articles_links]
        for article in articles_links:
            # print(article)
            data = scrape_content(link=article, category=k, subcategory=vals["title"])
            print(data)
            # data_list_dict.append(data)          
        
        # save_json("biblical_counseling", data_list_dict)
        print(articles_links)

{'title': '2024 Biblical Counseling Book of the Year', 'subtitle': ' ', 'author': 'ACBC', 'question': 'ACBC has chosen the 2024 biblical counseling book of the year! The winner was selected by a vote of our certified counselors.', 'body': 'The Association of Certified Biblical Counselors has chosen the 2024 biblical counseling book of the year. The winner was selected by a vote of ACBC’s certified counselors. These titles have served countless biblical counselors this year as we aim to present one another mature in Christ with the power of His all-sufficient Word,.\n\nWinner: \nA Call to Clarity: Critical Issues in Contemporary Biblical Counseling by Heath Lambert\nThe contemporary biblical counseling movement is facing days of significant challenges. Many voices with confused misunderstandings of biblical truth are seeking to redefine biblical counseling. They are attempting to blur the clear lines between approaches to counseling care that are uniquely biblical and those that are fun